<a href="https://colab.research.google.com/github/me1nna/nlp-sirius-tbank/blob/main/qa_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence_transformers==3.1.1 torch==2.4.1 faiss-gpu pyarrow==11.0.0 datasets==2.14.5

In [ ]:
import torch


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Загружаем модели ретривера и генератора

In [ ]:
from sentence_transformers import SentenceTransformer
from transformers import (
    AutoTokenizer,
    T5ForConditionalGeneration
)


retriever_model = SentenceTransformer("BAAI/bge-m3", device=device)
retriever_model.max_seq_length = 512

generator_checkpoint = 'hivaze/AAQG-QA-QG-FRED-T5-1.7B'
generator_tokenizer = AutoTokenizer.from_pretrained(generator_checkpoint)
generator_model = T5ForConditionalGeneration.from_pretrained(generator_checkpoint).to(device)

### Посмотрим на работу выбранного генератора, справляется ли он с задачей ответа на вопрос,  если ответ содержится в самом вопросе? (при условии контекста)

In [ ]:
QA_PROMPT = "Сгенерируй ответ на вопрос по тексту. Текст: '{context}'. Вопрос: '{question}'."

In [ ]:
from functools import partial


def generate_text(prompt, tokenizer, model, n=1, temperature=0.8, num_beams=3):
  encoded_input = tokenizer.encode_plus(prompt, return_tensors='pt')
  encoded_input = {k: v.to(model.device) for k, v in encoded_input.items()}

  resulted_tokens = model.generate(**encoded_input,
                                   max_new_tokens=64,
                                   do_sample=True,
                                   num_beams=num_beams,
                                   num_return_sequences=n,
                                   temperature=temperature,
                                   top_p=0.9,
                                   top_k=50)
  resulted_texts = tokenizer.batch_decode(resulted_tokens, skip_special_tokens=True)

  return resulted_texts


generate_text = partial(generate_text, tokenizer=generator_tokenizer, model=generator_model)

In [ ]:
test_context = "Путешественник Федор Конюхов и пилот Игорь Потапкин установили мировой рекорд высоты полета на паралёте, поднявшись на высоту 4728 метров — сайт Конюхова"

generate_text(QA_PROMPT.format(
  context=test_context,
  question='Как зовут Игоря Потапкина?'
), n=1)

['Игорь Потапкин']

### Загрузим датасет SberQuad

In [ ]:
from datasets import load_dataset


dataset = load_dataset("kuznetsoffandrey/sberquad")

### Создадим эмбеддинги для контекста. В качестве контекста берем столбец 'context' и удаляем повторения

In [ ]:
contexts = list(set(dataset['train']['context']))

### Реализуем разбение на чанки. Будем бить контексты на чанки, перед созданием эмбеддингов.

In [ ]:
import textwrap


def chunk_text(text, chunk_size=512):
    return textwrap.wrap(text, chunk_size)


chunked_contexts = []
for context in contexts:
    chunks = chunk_text(context, chunk_size=512)  # Например, 512 символов на чанк
    chunked_contexts.extend(chunks)

In [ ]:
len(chunked_contexts)

18991

In [ ]:
embeddings = retriever_model.encode(chunked_contexts, convert_to_tensor=True).detach().cpu().numpy()

### Создадим kNN

In [ ]:
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors(metric='cosine')
knn.fit(embeddings)

### Функция, для нахождения релевантного вопросу контекста с помощью kNN:

In [ ]:
def answer_question(question):
    question_embedding = retriever_model.encode([question], convert_to_tensor=True).detach().cpu().numpy()

    distances, indices = knn.kneighbors(question_embedding, n_neighbors=1)

    relevant_context = chunked_contexts[indices[0][0]]

    answer = generate_text(QA_PROMPT.format(
        context=relevant_context,
        question=question,
    ), n=1)

    return answer[0]


### Я — QA-система. Задавайте вопросы

In [ ]:
answer_question(input())

Какой город является столицой Франции, известный как Париж?


'Париж'

## Интересные и полезные дополнительные фичи

### 1. Функция для детекции **токсичности**

In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification
)

In [ ]:
toxity_checkpoint = 'cointegrated/rubert-tiny-toxicity'
toxicity_detection_tokenizer = AutoTokenizer.from_pretrained(toxity_checkpoint)
toxicity_detection_model = AutoModelForSequenceClassification.from_pretrained(toxity_checkpoint)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
toxicity_detection_model.to(device)

In [ ]:
def detect_toxicity(text):
    inputs = toxicity_detection_tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)

    with torch.no_grad():
        outputs = toxicity_detection_model(**inputs)

    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)
    toxicity_probability = probabilities[0][1].item()

    treshhold = 0.5

    return True if (toxicity_probability >= treshhold) else False

In [ ]:
text = "как зовут Васю Пупкина?"
text_toxic = "Да как зовут этого дурачка Васю Пупкина?"
detect_toxicity(text), detect_toxicity(text_toxic)

(False, True)

### 2. Проверка **орфографии**


#### Может быть особенно полезно, ведь правописание вопроса может оказаться далеко от идеала, а это может затруднить работу языковой модели.


In [ ]:
from transformers import (
    T5TokenizerFast,
    AutoModelForSeq2SeqLM
)

In [ ]:
spell_checkpoint = 'UrukHan/t5-russian-spell'
spell_checker_tokenizer = T5TokenizerFast.from_pretrained(spell_checkpoint)
spell_checker_model = AutoModelForSeq2SeqLM.from_pretrained(spell_checkpoint).to(device)

tokenizer_config.json:   0%|          | 0.00/1.94k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.63M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [ ]:
MAX_INPUT = 512

In [ ]:
def correct_spelling(input_sequences):
    task_prefix = "Spell correct: "  # Префикс задачи
    if type(input_sequences) != list:
        input_sequences = [input_sequences]

    encoded = spell_checker_tokenizer(
        [task_prefix + sequence for sequence in input_sequences],
        padding="longest",
        max_length=MAX_INPUT,
        truncation=True,
        return_tensors="pt",
    ).to(device)

    with torch.no_grad():
        predicts = spell_checker_model.generate(**encoded)

    corrected_texts = spell_checker_tokenizer.batch_decode(predicts, skip_special_tokens=True)

    return corrected_texts


In [ ]:
input_sequences = ['сеглдыя хороши ден', 'когд а вы прдет к нам в госи']
corrected_texts = correct_spelling(input_sequences)
print(corrected_texts)

['Сегодня хороший день.', 'Когда вы придете к нам в гости?']
